# Frequent itemset mining

- Student ID: 21127191
- Student name: Nguyễn Nhật Truyền

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example:

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29

30 31 32

33 34 35

36 37 38 39 40 41 42 43 44 45 46

38 39 47 48

38 39 48 49 50 51 52 53 54 55 56 57 58

32 41 59 60 61 62

3 39 48

63 64 65 66 67 68



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [51]:
from collections import defaultdict

### Read data
First we have to read data from database

In [52]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.

    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) #* Initialize a dictionary for storing support of items in I.
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) #* A python set is a native way for storing an itemset.
            for item in itemset:
                s[item]+=1     #* Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    return data, s

### Tree Projection

**Question 0: I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**Answer:**

In [53]:
def joinset(a,b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # a,b same brach -> a_k = b_k
    # Hint: this function will be called in generateSearchSpace method.:
    sa = set(a)
    sb = set(b)
    ret = sa.union(sb)
    ret = list(ret)
    ret.sort()
    return ret

class TP:
    def __init__ (self, data=None, s=None, minSup=None):
        self.data=data
        self.s={}


        for key, support in sorted(s.items(),key= lambda item: item[1]):
            self.s[key]=support
        # Why should we do this, answer it at the markdown below?

        self.minSup=minSup
        self.L={}  #Store frequent itemsets mined from database
        self.runAlgorithm()
    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree={}

        search_space={}
        for item, support in self.s.items():
            search_space[item]={}

            search_space[item]['itemset']=[item]
            '''
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned']=False


            search_space[item]['support']=support

            tree[item]={}
            '''
            Why should i store an additional tree (here it called tree)?
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for transaction in self.data.values():
          if set(itemset).issubset(set(transaction)):
            support += 1
        return support



    def prune(self,k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        #TODO
        pruned_items = []

        for item, info in search_space.items():
            if info['support'] >= self.minSup:
                self.L.setdefault(k, [])
                self.L[k].append((info['itemset'], info['support']))
            else:
                pruned_items.append(item)

        for item in pruned_items:
            search_space[item]['pruned'] = True




    def generateSearchSpace(self,k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items=list(tree.keys())

        ''' print search_space.keys() you will understand
         why we need an additional tree, '''
        l=len(items)
        self.prune(k, tree, search_space)
        if l==0: return   #Stop condition

        for i in range(l-1):
            a=items[i]
            if search_space[a]['pruned']: continue


            for j in range(i+1,l):
                b=items[j]

                search_space[a][b]={}
                tree[a][b]={}
                #TODO
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                #First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])

                #Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False
                search_space[a][b]['support'] = self.computeItemsetSupport(newset)
                tree[a][b] = search_space[a][b]
            #  Generate search_space for k+1-itemset
            search_space[a].pop('itemset')
            search_space[a].pop('pruned')
            search_space[a].pop('support')
            self.generateSearchSpace(k+1,tree[a],search_space[a])


    def runAlgorithm(self):
        tree,search_space=self.initialize() #generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)
    def miningResults(self):
        return self.L


Ok, let's test on a typical dataset `chess`.

In [54]:
transactions, freq= readData('chess.txt')

In [55]:
# Run and print result (better print format)
a=TP(data=transactions,s=freq, minSup=3000)
print(*a.miningResults().items(),sep="\n")

(1, [([48], 3013), ([56], 3021), ([66], 3021), ([34], 3040), ([62], 3060), ([7], 3076), ([36], 3099), ([60], 3149), ([40], 3170), ([29], 3181), ([52], 3185), ([58], 3195)])
(2, [([48, 52], 3002), ([48, 58], 3012), ([29, 56], 3006), ([52, 56], 3016), ([56, 58], 3020), ([60, 66], 3021), ([29, 66], 3013), ([52, 66], 3010), ([58, 66], 3020), ([34, 40], 3017), ([29, 34], 3036), ([34, 52], 3031), ([34, 58], 3039), ([60, 62], 3014), ([40, 62], 3045), ([29, 62], 3045), ([52, 62], 3049), ([58, 62], 3060), ([7, 60], 3031), ([7, 40], 3050), ([7, 29], 3069), ([7, 52], 3065), ([7, 58], 3075), ([36, 60], 3052), ([36, 40], 3073), ([29, 36], 3084), ([36, 52], 3088), ([36, 58], 3098), ([40, 60], 3124), ([29, 60], 3136), ([52, 60], 3138), ([58, 60], 3148), ([29, 40], 3155), ([40, 52], 3159), ([40, 58], 3169), ([29, 52], 3170), ([29, 58], 3180), ([52, 58], 3184)])
(3, [([48, 52, 58], 3001), ([29, 52, 56], 3001), ([29, 56, 58], 3005), ([52, 56, 58], 3015), ([29, 60, 66], 3013), ([52, 60, 66], 3010), ([58,

### Answer questions here:


**Question 1. Why should we sort all items in s by it's support and why we have to restored it to new artribute s?**

**Answer:**

*   Sorting all items in s by their support is done to prioritize the processing of items with higher support during the Apriori algorithm. This is a common optimization strategy in frequent itemset mining algorithms.
*   The reason for storing the sorted items in a new attribute self.s is to have a sorted representation of items based on their support. This sorted order is used when initializing the search space (tree and search_space structures). Sorting allows the algorithm to start with items that have lower support, making the search space generation and pruning more efficient.
*   The sorting step helps to organize the items in a way that facilitates early pruning of infrequent itemsets, leading to faster execution of the Apriori algorithm.


**Question 2. Python set does not remain elements order so we use a list to extend it easily when create new itemset but why we store itemset in data by a python set????**

**Answer:**

*   The reason for using a Python set in the data dictionary to store itemsets is to take advantage of the set's mathematical properties, specifically the ability to efficiently check for subset relationships.
*   When checking if one itemset is a subset of another, the order of elements does not matter, and using sets allows for quick and efficient subset checks. This is crucial when determining the support of an itemset within a transaction.

*   Beside, using a list to extend and create new itemsets (as seen in the joinset function) is helpful during the generation of candidate itemsets in the search space. Lists maintain the order of elements, which is important when creating new combinations of items. So, sets are used for efficient membership tests, while lists are used for maintaining order during the candidate generation process.





**Question 3.  After finish implementing the algorithm tell me why should you use this instead of delete item directly from search_space and tree.**

**Answer:**

*   Tree Projection is a technique used to avoid recomputing support counts for itemsets from scratch in each level of the search space. Instead of deleting items directly from search_space and tree and recomputing support for each iteration, Tree Projection leverages the support information already calculated in previous levels.
*   Deleting items directly from the structures would require re-evaluating the support of each itemset at every level of the search space. This can be computationally expensive, especially when dealing with large datasets and deep search spaces.

*   By using Tree Projection, the algorithm takes advantage of the fact that the support of an itemset can be projected down the tree from higher levels to lower levels. This reduces the computational cost and speeds up the execution of the Apriori algorithm.

*   Tree Projection is an optimization that helps avoid redundant calculations and improves the efficiency of the frequent itemset mining process.





# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). You can download dataset from here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/churn.txt. Write your report and implementation below.

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  
More information about `churn` here: http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir4/Churn.pdf)*

**TODO:**

# 4 References

http://ce.sharif.edu/courses/85-86/1/ce925/assignments/files/assignDir2/ProjectDefinition1.pdf

https://cs.wmich.edu/~alfuqaha/summer14/cs6530/lectures/AssociationAnalysis-Part1.pdf

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
